In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import warnings

warnings.filterwarnings(action = 'ignore')

In [2]:
df = pd.read_csv('../data/final_DE.csv')
pd.set_option('display.max_columns',25)
df

,Unnamed: 0,GU,DONG_CODE,DONG,SUBWAY_NUM,STARBUCKS_NUM,SPORT_NUM,SAFE_DLVR_NUM,MC_NUM,KINDER_NUM,KIDS_NUM,GYM_NUM,GOLF_NUM,ACADEMY_NUM,ANIMAL_HSPT_NUM,BIKE_NUM,CAFE,CAR_SHR_NUM,CCTV_NUM,CHILD_MED_NUM,CON_NUM,DPTM_NUM
0,0,종로구,1111010100,청운동,0,0,0,0,0,0,0,0,0,2,0,2,8,0,12,0,1,0
1,1,종로구,1111010200,신교동,0,0,0,0,0,0,0,0,0,5,2,0,1,0,14,0,0,0
2,2,종로구,1111010300,궁정동,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,3,종로구,1111010400,효자동,0,0,0,0,0,0,0,0,0,0,0,1,5,0,0,0,0,0
4,4,종로구,1111010500,창성동,0,0,0,0,0,0,0,0,0,1,0,1,3,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,462,강동구,1174010700,암사동,1,1,0,2,0,30,4,16,8,197,10,11,70,4,133,11,33,0
463,463,강동구,1174010900,천호동,1,4,0,3,1,70,6,14,7,124,8,15,131,17,266,12,68,1
464,464,강동구,1174010800,성내동,4,4,0,2,0,50,2,30,14,190,16,18,143,20,156,12,58,0
465,465,강동구,1174010500,길동,0,2,0,1,0,20,1,18,9,100,16,9,63,10,119,10,38,0


In [3]:
df.rename(columns={'Unnamed: 0':'index'},inplace=True)
df.set_index('index',inplace=True)
df

,GU,DONG_CODE,DONG,SUBWAY_NUM,STARBUCKS_NUM,SPORT_NUM,SAFE_DLVR_NUM,MC_NUM,KINDER_NUM,KIDS_NUM,GYM_NUM,GOLF_NUM,ACADEMY_NUM,ANIMAL_HSPT_NUM,BIKE_NUM,CAFE,CAR_SHR_NUM,CCTV_NUM,CHILD_MED_NUM,CON_NUM,DPTM_NUM
index,,,,,,,,,,,,,,,,,,,,,
0,종로구,1111010100,청운동,0,0,0,0,0,0,0,0,0,2,0,2,8,0,12,0,1,0
1,종로구,1111010200,신교동,0,0,0,0,0,0,0,0,0,5,2,0,1,0,14,0,0,0
2,종로구,1111010300,궁정동,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,종로구,1111010400,효자동,0,0,0,0,0,0,0,0,0,0,0,1,5,0,0,0,0,0
4,종로구,1111010500,창성동,0,0,0,0,0,0,0,0,0,1,0,1,3,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,강동구,1174010700,암사동,1,1,0,2,0,30,4,16,8,197,10,11,70,4,133,11,33,0
463,강동구,1174010900,천호동,1,4,0,3,1,70,6,14,7,124,8,15,131,17,266,12,68,1
464,강동구,1174010800,성내동,4,4,0,2,0,50,2,30,14,190,16,18,143,20,156,12,58,0


In [4]:
std = pd.read_excel('../data/한국행정구역분류_2022.10.1.기준_20221006123917.xlsx',sheet_name='행정동 및 법정동코드와의 연계표')
std

,한국행정구역분류와 행정동(행정기관) 및 법정동코드 연계표(2022.10.1.기준),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,시도,시군구,행정구역명,행정동\n(행정기관명),법정동,행정구역분류\n(개정_8자리),행정구역분류\n(7자리),행정기관코드,행정기관 생성일,법정동코드,법정동 관할구역\n분할여부,행정동 영문명칭,비고
1,서울특별시,서울특별시,서울특별시,서울특별시,서울특별시,11,11,1100000000,19880423,1100000000,NaN,Seoul,NaN
2,서울특별시,종로구,종로구,종로구,종로구,11010,11010,1111000000,19880423,1111000000,NaN,Jongno-gu,NaN
3,서울특별시,종로구,청운효자동,청운효자동,청운동,11010720,1101072,1111051500,20081101,1111010100,NaN,Cheongunhyoja-dong,NaN
4,서울특별시,종로구,청운효자동,청운효자동,신교동,11010720,1101072,1111051500,20081101,1111010200,NaN,Cheongunhyoja-dong,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21718,제주특별자치도,서귀포시,중문동,중문동,대포동,39020610,3902061,5013061000,20060701,5013011400,NaN,Jungmun-dong,NaN
21719,제주특별자치도,서귀포시,중문동,중문동,하원동,39020610,3902061,5013061000,20060701,5013011800,NaN,Jungmun-dong,NaN
21720,제주특별자치도,서귀포시,예래동,예래동,색달동,39020620,3902062,5013062000,20060701,5013011900,NaN,Yerae-dong,NaN
21721,제주특별자치도,서귀포시,예래동,예래동,상예동,39020620,3902062,5013062000,20060701,5013012000,NaN,Yerae-dong,NaN


In [5]:
std.columns = std.loc[0]
std = std[1:]
std

,시도,시군구,행정구역명,행정동\n(행정기관명),법정동,행정구역분류\n(개정_8자리),행정구역분류\n(7자리),행정기관코드,행정기관 생성일,법정동코드,법정동 관할구역\n분할여부,행정동 영문명칭,비고
1,서울특별시,서울특별시,서울특별시,서울특별시,서울특별시,11,11,1100000000,19880423,1100000000,NaN,Seoul,NaN
2,서울특별시,종로구,종로구,종로구,종로구,11010,11010,1111000000,19880423,1111000000,NaN,Jongno-gu,NaN
3,서울특별시,종로구,청운효자동,청운효자동,청운동,11010720,1101072,1111051500,20081101,1111010100,NaN,Cheongunhyoja-dong,NaN
4,서울특별시,종로구,청운효자동,청운효자동,신교동,11010720,1101072,1111051500,20081101,1111010200,NaN,Cheongunhyoja-dong,NaN
5,서울특별시,종로구,청운효자동,청운효자동,궁정동,11010720,1101072,1111051500,20081101,1111010300,NaN,Cheongunhyoja-dong,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21718,제주특별자치도,서귀포시,중문동,중문동,대포동,39020610,3902061,5013061000,20060701,5013011400,NaN,Jungmun-dong,NaN
21719,제주특별자치도,서귀포시,중문동,중문동,하원동,39020610,3902061,5013061000,20060701,5013011800,NaN,Jungmun-dong,NaN
21720,제주특별자치도,서귀포시,예래동,예래동,색달동,39020620,3902062,5013062000,20060701,5013011900,NaN,Yerae-dong,NaN
21721,제주특별자치도,서귀포시,예래동,예래동,상예동,39020620,3902062,5013062000,20060701,5013012000,NaN,Yerae-dong,NaN


In [6]:
std_seoul = std[std['시도'] == '서울특별시']
std_df = std_seoul[['시군구','행정동\n(행정기관명)','법정동','행정기관코드','법정동코드','법정동 관할구역\n분할여부']][1:]
std_df.columns = ['시군구','행정동','법정동','행정기관코드','법정동코드','분할여부']
std_df

,시군구,행정동,법정동,행정기관코드,법정동코드,분할여부
2,종로구,종로구,종로구,1111000000,1111000000,NaN
3,종로구,청운효자동,청운동,1111051500,1111010100,NaN
4,종로구,청운효자동,신교동,1111051500,1111010200,NaN
5,종로구,청운효자동,궁정동,1111051500,1111010300,NaN
6,종로구,청운효자동,효자동,1111051500,1111010400,NaN
...,...,...,...,...,...,...
763,강동구,성내제2동,성내동,1174065000,1174010800,분할연계
764,강동구,성내제3동,성내동,1174066000,1174010800,분할연계
765,강동구,길동,길동,1174068500,1174010500,NaN
766,강동구,둔촌제1동,둔촌동,1174069000,1174010600,분할연계


In [7]:
std_df = std_df[std_df['시군구'] != std_df['행정동']]
std_df

,시군구,행정동,법정동,행정기관코드,법정동코드,분할여부
3,종로구,청운효자동,청운동,1111051500,1111010100,NaN
4,종로구,청운효자동,신교동,1111051500,1111010200,NaN
5,종로구,청운효자동,궁정동,1111051500,1111010300,NaN
6,종로구,청운효자동,효자동,1111051500,1111010400,NaN
7,종로구,청운효자동,창성동,1111051500,1111010500,NaN
...,...,...,...,...,...,...
763,강동구,성내제2동,성내동,1174065000,1174010800,분할연계
764,강동구,성내제3동,성내동,1174066000,1174010800,분할연계
765,강동구,길동,길동,1174068500,1174010500,NaN
766,강동구,둔촌제1동,둔촌동,1174069000,1174010600,분할연계


In [8]:
area_df = pd.read_csv('../data/AREA_DONG.csv')
del(area_df['SI_DO'])
del(area_df['GU'])
area_df

,DONG,AREA
0,청운동,"797,032.5"
1,신교동,"99,144.0"
2,궁정동,"73,042.7"
3,효자동,"59,135.9"
4,창성동,"59,706.1"
...,...,...
462,둔촌동,"2,461,511.8"
463,암사동,"4,711,364.2"
464,성내동,"1,958,987.6"
465,천호동,"3,065,543.3"


In [9]:
population_df = pd.read_csv('../data/POPU_DONG.csv')
del(population_df['DONG_CODE'])
del(population_df['SI_DO'])
del(population_df['GU'])
population_df

,DONG,NUM_POPU
0,청운동,2330
1,신교동,1393
2,궁정동,179
3,효자동,572
4,창성동,439
...,...,...
462,둔촌동,24311
463,암사동,66490
464,성내동,65873
465,천호동,85928


In [10]:
area_df['idx'] = [i for i in range(467)]
area_df.set_index('idx',inplace=True)
population_df['idx'] = [i for i in range(467)]
population_df.set_index('idx',inplace=True)

In [11]:
density_df = pd.merge(area_df,population_df,how='outer',on='idx')
density_df

,DONG_x,AREA,DONG_y,NUM_POPU
idx,,,,
0,청운동,"797,032.5",청운동,2330
1,신교동,"99,144.0",신교동,1393
2,궁정동,"73,042.7",궁정동,179
3,효자동,"59,135.9",효자동,572
4,창성동,"59,706.1",창성동,439
...,...,...,...,...
462,둔촌동,"2,461,511.8",둔촌동,24311
463,암사동,"4,711,364.2",암사동,66490
464,성내동,"1,958,987.6",성내동,65873


In [12]:
del(density_df['DONG_y'])
density_df.columns = ['법정동','면적','인구']
density_df

,법정동,면적,인구
idx,,,
0,청운동,"797,032.5",2330
1,신교동,"99,144.0",1393
2,궁정동,"73,042.7",179
3,효자동,"59,135.9",572
4,창성동,"59,706.1",439
...,...,...,...
462,둔촌동,"2,461,511.8",24311
463,암사동,"4,711,364.2",66490
464,성내동,"1,958,987.6",65873


In [13]:
area = density_df['면적'].tolist()
population = density_df['인구'].tolist()
for i in range(len(area)):
    area[i] = float(area[i].replace(',',''))
density = []
for i in range(len(density_df.index)):
    dense = population[i] / area[i]
    density.append(dense)
density_df['인구밀도'] = density
density_df

,법정동,면적,인구,인구밀도
idx,,,,
0,청운동,"797,032.5",2330,0.002923
1,신교동,"99,144.0",1393,0.014050
2,궁정동,"73,042.7",179,0.002451
3,효자동,"59,135.9",572,0.009673
4,창성동,"59,706.1",439,0.007353
...,...,...,...,...
462,둔촌동,"2,461,511.8",24311,0.009876
463,암사동,"4,711,364.2",66490,0.014113
464,성내동,"1,958,987.6",65873,0.033626


In [14]:
tmp_df = df.copy()
tmp_df

,GU,DONG_CODE,DONG,SUBWAY_NUM,STARBUCKS_NUM,SPORT_NUM,SAFE_DLVR_NUM,MC_NUM,KINDER_NUM,KIDS_NUM,GYM_NUM,GOLF_NUM,ACADEMY_NUM,ANIMAL_HSPT_NUM,BIKE_NUM,CAFE,CAR_SHR_NUM,CCTV_NUM,CHILD_MED_NUM,CON_NUM,DPTM_NUM
index,,,,,,,,,,,,,,,,,,,,,
0,종로구,1111010100,청운동,0,0,0,0,0,0,0,0,0,2,0,2,8,0,12,0,1,0
1,종로구,1111010200,신교동,0,0,0,0,0,0,0,0,0,5,2,0,1,0,14,0,0,0
2,종로구,1111010300,궁정동,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,종로구,1111010400,효자동,0,0,0,0,0,0,0,0,0,0,0,1,5,0,0,0,0,0
4,종로구,1111010500,창성동,0,0,0,0,0,0,0,0,0,1,0,1,3,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,강동구,1174010700,암사동,1,1,0,2,0,30,4,16,8,197,10,11,70,4,133,11,33,0
463,강동구,1174010900,천호동,1,4,0,3,1,70,6,14,7,124,8,15,131,17,266,12,68,1
464,강동구,1174010800,성내동,4,4,0,2,0,50,2,30,14,190,16,18,143,20,156,12,58,0


In [15]:
num = std_df[std_df['분할여부'] == '분할연계']['법정동코드'].value_counts()
num_df= pd.DataFrame(num)
num_df

,법정동코드
1162010200,11
1162010100,9
1135010500,8
1150010300,8
1156013200,7
...,...
1150010600,2
1123011000,2
1126010200,2
1126010300,2


In [22]:
std_df[std_df['법정동코드'] == num_df.index[0]]

,시군구,행정동,법정동,행정기관코드,법정동코드,분할여부
650,관악구,서원동,신림동,1162064500,1162010200,분할연계
651,관악구,신원동,신림동,1162065500,1162010200,분할연계
652,관악구,서림동,신림동,1162066500,1162010200,분할연계
653,관악구,신사동,신림동,1162068500,1162010200,분할연계
654,관악구,신림동,신림동,1162069500,1162010200,분할연계
655,관악구,난향동,신림동,1162071500,1162010200,분할연계
656,관악구,조원동,신림동,1162072500,1162010200,분할연계
657,관악구,대학동,신림동,1162073500,1162010200,분할연계
658,관악구,삼성동,신림동,1162074500,1162010200,분할연계
659,관악구,미성동,신림동,1162076500,1162010200,분할연계


In [41]:
for x in num_df.index:
    y = tmp_df[tmp_df['DONG_CODE'] == x] # x : 법정동 , y :법정동에 대한 데이터
    k = num_df[num_df.index == x]['법정동코드'].tolist() # k : 해당 법정동코드에 속하는 법정동 수
    z = y[y.columns[3:]]
    for i in range(len(z.columns)):
        y[y.columns[i+3]] = z[z.columns[i]] / k # 각 법정동 데이터의 수치를 법정동 수로 나누어 줌
        # 각 행정동에 동일하게 분배
    tmp_df[tmp_df['DONG_CODE'] == x] = y
tmp_df

,GU,DONG_CODE,DONG,SUBWAY_NUM,STARBUCKS_NUM,SPORT_NUM,SAFE_DLVR_NUM,MC_NUM,KINDER_NUM,KIDS_NUM,GYM_NUM,GOLF_NUM,ACADEMY_NUM,ANIMAL_HSPT_NUM,BIKE_NUM,CAFE,CAR_SHR_NUM,CCTV_NUM,CHILD_MED_NUM,CON_NUM,DPTM_NUM
index,,,,,,,,,,,,,,,,,,,,,
0,종로구,1111010100,청운동,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,2.000000,8.000000,0.000000,12.000000,0.000000,1.000000,0.000000
1,종로구,1111010200,신교동,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2.000000,0.000000,1.000000,0.000000,14.000000,0.000000,0.000000,0.000000
2,종로구,1111010300,궁정동,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,종로구,1111010400,효자동,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,종로구,1111010500,창성동,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,3.000000,0.000000,0.000000,2.000000,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,강동구,1174010700,암사동,0.037037,0.037037,0.0,0.074074,0.000000,1.111111,0.148148,0.592593,0.296296,7.296296,0.370370,0.407407,2.592593,0.148148,4.925926,0.407407,1.222222,0.000000
463,강동구,1174010900,천호동,0.037037,0.148148,0.0,0.111111,0.037037,2.592593,0.222222,0.518519,0.259259,4.592593,0.296296,0.555556,4.851852,0.629630,9.851852,0.444444,2.518519,0.037037
464,강동구,1174010800,성내동,0.148148,0.148148,0.0,0.074074,0.000000,1.851852,0.074074,1.111111,0.518519,7.037037,0.592593,0.666667,5.296296,0.740741,5.777778,0.444444,2.148148,0.000000


In [25]:
tmp_df[tmp_df['DONG'] == '신림동']

,GU,DONG_CODE,DONG,SUBWAY_NUM,STARBUCKS_NUM,SPORT_NUM,SAFE_DLVR_NUM,MC_NUM,KINDER_NUM,KIDS_NUM,GYM_NUM,GOLF_NUM,ACADEMY_NUM,ANIMAL_HSPT_NUM,BIKE_NUM,CAFE,CAR_SHR_NUM,CCTV_NUM,CHILD_MED_NUM,CON_NUM,DPTM_NUM
index,,,,,,,,,,,,,,,,,,,,,
420,관악구,1162010200,신림동,0.090909,0.363636,0.0,0.909091,0.090909,9.090909,0.181818,6.0,1.272727,27.090909,2.363636,4.181818,27.363636,3.636364,42.454545,11.818182,14.545455,0.0


In [26]:
def dong_divide(df, std_df):
    num = std_df[std_df['분할여부'] == '분할연계']['법정동코드'].value_counts()
    num_df= pd.DataFrame(num)
    for x in num_df.index:
        y = df[df['DONG_CODE'] == x] # x : 법정동 , y :법정동에 대한 데이터
        k = num_df[num_df.index == x]['법정동코드'].tolist() # k : 해당 법정동코드에 속하는 법정동 수
        z = y[y.columns[3:]]
        for i in range(len(z.columns)):
            y[y.columns[i+3]] = z[z.columns[i]] / k
        df[df['DONG_CODE'] == x] = y
    return df

In [27]:
sample_df = dong_divide(tmp_df,std_df)
sample_df

,GU,DONG_CODE,DONG,SUBWAY_NUM,STARBUCKS_NUM,SPORT_NUM,SAFE_DLVR_NUM,MC_NUM,KINDER_NUM,KIDS_NUM,GYM_NUM,GOLF_NUM,ACADEMY_NUM,ANIMAL_HSPT_NUM,BIKE_NUM,CAFE,CAR_SHR_NUM,CCTV_NUM,CHILD_MED_NUM,CON_NUM,DPTM_NUM
index,,,,,,,,,,,,,,,,,,,,,
0,종로구,1111010100,청운동,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,2.000000,8.000000,0.000000,12.000000,0.000000,1.000000,0.000000
1,종로구,1111010200,신교동,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2.000000,0.000000,1.000000,0.000000,14.000000,0.000000,0.000000,0.000000
2,종로구,1111010300,궁정동,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,종로구,1111010400,효자동,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,종로구,1111010500,창성동,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,3.000000,0.000000,0.000000,2.000000,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,강동구,1174010700,암사동,0.111111,0.111111,0.0,0.222222,0.000000,3.333333,0.444444,1.777778,0.888889,21.888889,1.111111,1.222222,7.777778,0.444444,14.777778,1.222222,3.666667,0.000000
463,강동구,1174010900,천호동,0.111111,0.444444,0.0,0.333333,0.111111,7.777778,0.666667,1.555556,0.777778,13.777778,0.888889,1.666667,14.555556,1.888889,29.555556,1.333333,7.555556,0.111111
464,강동구,1174010800,성내동,0.444444,0.444444,0.0,0.222222,0.000000,5.555556,0.222222,3.333333,1.555556,21.111111,1.777778,2.000000,15.888889,2.222222,17.333333,1.333333,6.444444,0.000000


In [28]:
area_code = std_df['행정기관코드'].unique()
area_code

array([1111051500, 1111053000, 1111054000, 1111055000, 1111056000,
       1111057000, 1111058000, 1111060000, 1111061500, 1111063000,
       1111064000, 1111065000, 1111067000, 1111068000, 1111069000,
       1111070000, 1111071000, 1114052000, 1114054000, 1114055000,
       1114057000, 1114058000, 1114059000, 1114060500, 1114061500,
       1114062500, 1114063500, 1114064500, 1114065000, 1114066500,
       1114067000, 1114068000, 1117051000, 1117052000, 1117053000,
       1117055500, 1117056000, 1117057000, 1117058000, 1117059000,
       1117062500, 1117063000, 1117064000, 1117065000, 1117066000,
       1117068500, 1117069000, 1117070000, 1120052000, 1120053500,
       1120054000, 1120055000, 1120056000, 1120057000, 1120058000,
       1120059000, 1120061500, 1120062000, 1120064500, 1120065000,
       1120066000, 1120067000, 1120069000, 1120072000, 1120079000,
       1121571000, 1121573000, 1121574000, 1121575000, 1121576000,
       1121577000, 1121578000, 1121581000, 1121582000, 1121583

In [29]:
tmp = std_df[std_df['행정기관코드'] == area_code[0]]
tmp

,시군구,행정동,법정동,행정기관코드,법정동코드,분할여부
3,종로구,청운효자동,청운동,1111051500,1111010100,NaN
4,종로구,청운효자동,신교동,1111051500,1111010200,NaN
5,종로구,청운효자동,궁정동,1111051500,1111010300,NaN
6,종로구,청운효자동,효자동,1111051500,1111010400,NaN
7,종로구,청운효자동,창성동,1111051500,1111010500,NaN
8,종로구,청운효자동,통인동,1111051500,1111010800,NaN
9,종로구,청운효자동,누상동,1111051500,1111010900,NaN
10,종로구,청운효자동,누하동,1111051500,1111011000,NaN
11,종로구,청운효자동,옥인동,1111051500,1111011100,NaN
12,종로구,청운효자동,세종로,1111051500,1111011900,분할연계


In [30]:
code_list = tmp['법정동코드'].tolist()

In [31]:
x = pd.DataFrame(columns=tmp_df.columns)
for i in range(len(code_list)):
    y = tmp_df[tmp_df['DONG_CODE'] == code_list[i]]
    x = pd.concat([x,y])   
x

,GU,DONG_CODE,DONG,SUBWAY_NUM,STARBUCKS_NUM,SPORT_NUM,SAFE_DLVR_NUM,MC_NUM,KINDER_NUM,KIDS_NUM,GYM_NUM,GOLF_NUM,ACADEMY_NUM,ANIMAL_HSPT_NUM,BIKE_NUM,CAFE,CAR_SHR_NUM,CCTV_NUM,CHILD_MED_NUM,CON_NUM,DPTM_NUM
0,종로구,1111010100,청운동,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.000000,8.000000,0.000000,12.000000,0.000000,1.0,0.0
1,종로구,1111010200,신교동,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,2.0,0.000000,1.000000,0.000000,14.000000,0.000000,0.0,0.0
2,종로구,1111010300,궁정동,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.0
3,종로구,1111010400,효자동,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,5.000000,0.000000,0.000000,0.000000,0.0,0.0
4,종로구,1111010500,창성동,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.000000,3.000000,0.000000,0.000000,2.000000,1.0,0.0
5,종로구,1111010800,통인동,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,2.0,0.000000,4.000000,0.000000,0.000000,0.000000,0.0,0.0
6,종로구,1111010900,누상동,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,1.000000,0.000000,10.000000,0.000000,0.0,0.0
7,종로구,1111011000,누하동,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.000000,8.000000,0.000000,3.000000,2.000000,0.0,0.0
8,종로구,1111011100,옥인동,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.000000,5.000000,0.000000,15.000000,3.000000,0.0,0.0
9,종로구,1111011900,세종로,0.111111,0.111111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.555556,1.222222,0.333333,0.888889,0.111111,0.0,0.0


In [32]:
n = tmp.iloc[0][0:2].tolist()

In [33]:
for col in x.columns[3:]:
    n.append(sum(x[col]))
n

['종로구',
 '청운효자동',
 0.1111111111111111,
 0.1111111111111111,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 18.0,
 4.0,
 4.555555555555555,
 37.22222222222222,
 0.3333333333333333,
 54.888888888888886,
 7.111111111111111,
 2.0,
 0.0]

In [ ]:
x.columns[3:]

In [ ]:
sample_list = []
for i in range(len(area_code)):
    tmp = std_df[std_df['행정기관코드'] == area_code[i]]
    
    code_list = tmp['법정동코드'].tolist()
    x = pd.DataFrame(columns=tmp_df.columns)
    for i in range(len(code_list)):
        y = tmp_df[tmp_df['DONG_CODE'] == code_list[i]]
        x = pd.concat([x,y])   
    
    n = tmp.iloc[i][0:2].tolist()
    for col in x.columns[3:]:
        n.append(sum(x[col]))
    sample_list.append(n)
sample_list

In [ ]:
s = pd.DataFrame(sample_list,columns=['구이름','행정동','SUBWAY_NUM', 'STARBUCKS_NUM', 'SPORT_NUM', 'SAFE_DLVR_NUM', 'MC_NUM',
       'KINDER_NUM', 'KIDS_NUM', 'GYM_NUM', 'GOLF_NUM', 'ACADEMY_NUM',
       'ANIMAL_HSPT_NUM', 'BIKE_NUM', 'CAFE', 'CAR_SHR_NUM', 'CCTV_NUM',
       'CHILD_MED_NUM', 'CON_NUM', 'DPTM_NUM'])
s

In [ ]:
s['행정동'].unique()

In [ ]:
s.to_csv('행정동_데이터.csv',mode='w')

In [34]:
df[df['DONG'] == '성내동']

,GU,DONG_CODE,DONG,SUBWAY_NUM,STARBUCKS_NUM,SPORT_NUM,SAFE_DLVR_NUM,MC_NUM,KINDER_NUM,KIDS_NUM,GYM_NUM,GOLF_NUM,ACADEMY_NUM,ANIMAL_HSPT_NUM,BIKE_NUM,CAFE,CAR_SHR_NUM,CCTV_NUM,CHILD_MED_NUM,CON_NUM,DPTM_NUM
index,,,,,,,,,,,,,,,,,,,,,
464,강동구,1174010800,성내동,4,4,0,2,0,50,2,30,14,190,16,18,143,20,156,12,58,0


In [35]:
std_df[std_df['법정동코드'] == 1174010800]

,시군구,행정동,법정동,행정기관코드,법정동코드,분할여부
762,강동구,성내제1동,성내동,1174064000,1174010800,분할연계
763,강동구,성내제2동,성내동,1174065000,1174010800,분할연계
764,강동구,성내제3동,성내동,1174066000,1174010800,분할연계


In [39]:
num_df[num_df.index == 1174010800]['법정동코드'].tolist()

[3]

In [43]:
for x in num_df.index:
    y = tmp_df[tmp_df['DONG_CODE'] == x] # x : 법정동 , y :법정동에 대한 데이터
    k = num_df[num_df.index == x]['법정동코드'].tolist() # k : 해당 법정동코드에 속하는 법정동 수
    z = y[y.columns[3:]]
    for i in range(len(z.columns)):
        y[y.columns[i+3]] = z[z.columns[i]] / k # 각 법정동 데이터의 수치를 법정동 수로 나누어 줌
        # 각 행정동에 동일하게 분배
#     tmp_df[tmp_df['DONG_CODE'] == x] = y
# tmp_df
    print(y)

        GU   DONG_CODE DONG  SUBWAY_NUM  STARBUCKS_NUM  SPORT_NUM  \
index                                                               
420    관악구  1162010200  신림동    0.000068       0.000273        0.0   

       SAFE_DLVR_NUM    MC_NUM  KINDER_NUM  KIDS_NUM   GYM_NUM  GOLF_NUM  \
index                                                                      
420         0.000683  0.000068     0.00683  0.000137  0.004508  0.000956   

       ACADEMY_NUM  ANIMAL_HSPT_NUM  BIKE_NUM      CAFE  CAR_SHR_NUM  \
index                                                                  
420       0.020354         0.001776  0.003142  0.020559     0.002732   

       CCTV_NUM  CHILD_MED_NUM   CON_NUM  DPTM_NUM  
index                                               
420    0.031897       0.008879  0.010928       0.0  
        GU   DONG_CODE DONG  SUBWAY_NUM  STARBUCKS_NUM  SPORT_NUM  \
index                                                               
418    관악구  1162010100  봉천동    0.000457       0.0

        GU   DONG_CODE DONG  SUBWAY_NUM  STARBUCKS_NUM  SPORT_NUM  \
index                                                               
216    광진구  1121510100  중곡동         0.0       0.007812        0.0   

       SAFE_DLVR_NUM  MC_NUM  KINDER_NUM  KIDS_NUM   GYM_NUM  GOLF_NUM  \
index                                                                    
216         0.019531     0.0    0.078125  0.015625  0.105469  0.039062   

       ACADEMY_NUM  ANIMAL_HSPT_NUM  BIKE_NUM      CAFE  CAR_SHR_NUM  \
index                                                                  
216       0.554688         0.054688  0.039062  0.414062      0.03125   

       CCTV_NUM  CHILD_MED_NUM   CON_NUM  DPTM_NUM  
index                                               
216    0.546875       0.144531  0.199219       0.0  
        GU   DONG_CODE DONG  SUBWAY_NUM  STARBUCKS_NUM  SPORT_NUM  \
index                                                               
411    동작구  1159010200  상도동    0.007812       0.011719 

366    3.567901        1.62963  0.432099       0.0  
        GU   DONG_CODE DONG  SUBWAY_NUM  STARBUCKS_NUM  SPORT_NUM  \
index                                                               
9      종로구  1111011900  세종로    0.012346       0.012346        0.0   

       SAFE_DLVR_NUM  MC_NUM  KINDER_NUM  KIDS_NUM  GYM_NUM  GOLF_NUM  \
index                                                                   
9                0.0     0.0         0.0       0.0      0.0       0.0   

       ACADEMY_NUM  ANIMAL_HSPT_NUM  BIKE_NUM      CAFE  CAR_SHR_NUM  \
index                                                                  
9              0.0              0.0  0.061728  0.135802     0.037037   

       CCTV_NUM  CHILD_MED_NUM  CON_NUM  DPTM_NUM  
index                                              
9      0.098765       0.012346      0.0       0.0  
        GU   DONG_CODE DONG  SUBWAY_NUM  STARBUCKS_NUM  SPORT_NUM  \
index                                                               
322    마

189           0.5625   0.4375       0.0  
        GU   DONG_CODE  DONG  SUBWAY_NUM  STARBUCKS_NUM  SPORT_NUM  \
index                                                                
190    용산구  1117013000  이태원동      0.0625          0.125        0.0   

       SAFE_DLVR_NUM  MC_NUM  KINDER_NUM  KIDS_NUM  GYM_NUM  GOLF_NUM  \
index                                                                   
190           0.0625     0.0        1.25       0.0   0.5625    0.0625   

       ACADEMY_NUM  ANIMAL_HSPT_NUM  BIKE_NUM   CAFE  CAR_SHR_NUM  CCTV_NUM  \
index                                                                         
190          0.625            0.375     0.375  2.625        0.375    3.0625   

       CHILD_MED_NUM  CON_NUM  DPTM_NUM  
index                                    
190           0.3125   1.0625       0.0  
        GU   DONG_CODE   DONG  SUBWAY_NUM  STARBUCKS_NUM  SPORT_NUM  \
index                                                                 
197    성동구  112001020

447            0.875    0.625       0.0  
       GU   DONG_CODE   DONG  SUBWAY_NUM  STARBUCKS_NUM  SPORT_NUM  \
index                                                                
103    중구  1114012200  회현동2가         0.0            0.0        0.0   

       SAFE_DLVR_NUM  MC_NUM  KINDER_NUM  KIDS_NUM  GYM_NUM  GOLF_NUM  \
index                                                                   
103              0.0     0.0         0.0       0.0   0.0625       0.0   

       ACADEMY_NUM  ANIMAL_HSPT_NUM  BIKE_NUM  CAFE  CAR_SHR_NUM  CCTV_NUM  \
index                                                                        
103            0.0              0.0    0.0625   0.5          0.0    0.5625   

       CHILD_MED_NUM  CON_NUM  DPTM_NUM  
index                                    
103           0.0625    0.125       0.0  
        GU   DONG_CODE   DONG  SUBWAY_NUM  STARBUCKS_NUM  SPORT_NUM  \
index                                                                 
210    성동구  1120011400  

235            1.875   2.1875       0.0  
        GU   DONG_CODE DONG  SUBWAY_NUM  STARBUCKS_NUM  SPORT_NUM  \
index                                                               
365    구로구  1153010600  고척동         0.0         0.0625      0.125   

       SAFE_DLVR_NUM  MC_NUM  KINDER_NUM  KIDS_NUM  GYM_NUM  GOLF_NUM  \
index                                                                   
365            0.125  0.0625        6.25    0.1875   0.8125     0.875   

       ACADEMY_NUM  ANIMAL_HSPT_NUM  BIKE_NUM    CAFE  CAR_SHR_NUM  CCTV_NUM  \
index                                                                          
365         5.8125            0.625     0.625  2.9375       0.4375    19.625   

       CHILD_MED_NUM  CON_NUM  DPTM_NUM  
index                                    
365           5.3125   2.0625       0.0  
         GU   DONG_CODE   DONG  SUBWAY_NUM  STARBUCKS_NUM  SPORT_NUM  \
index                                                                  
402    영등포구  115601

In [44]:
df.loc[420]

GU                        관악구
DONG_CODE          1162010200
DONG                      신림동
SUBWAY_NUM                  1
STARBUCKS_NUM               4
SPORT_NUM                   0
SAFE_DLVR_NUM              10
MC_NUM                      1
KINDER_NUM                100
KIDS_NUM                    2
GYM_NUM                    66
GOLF_NUM                   14
ACADEMY_NUM               298
ANIMAL_HSPT_NUM            26
BIKE_NUM                   46
CAFE                      301
CAR_SHR_NUM                40
CCTV_NUM                  467
CHILD_MED_NUM             130
CON_NUM                   160
DPTM_NUM                    0
Name: 420, dtype: object

In [45]:
df[df['DONG'] == '세종로']

,GU,DONG_CODE,DONG,SUBWAY_NUM,STARBUCKS_NUM,SPORT_NUM,SAFE_DLVR_NUM,MC_NUM,KINDER_NUM,KIDS_NUM,GYM_NUM,GOLF_NUM,ACADEMY_NUM,ANIMAL_HSPT_NUM,BIKE_NUM,CAFE,CAR_SHR_NUM,CCTV_NUM,CHILD_MED_NUM,CON_NUM,DPTM_NUM
index,,,,,,,,,,,,,,,,,,,,,
9,종로구,1111011900,세종로,1,1,0,0,0,0,0,0,0,0,0,5,11,3,8,1,0,0


In [49]:
std_df[std_df['법정동'] == '세종로']['법정동코드'].iloc[0]

1111011900

In [50]:
num_df[num_df.index == std_df[std_df['법정동'] == '세종로']['법정동코드'].iloc[0]]['법정동코드'].tolist()

[3]